## Data conversion for a tensile test experiment sample

This notebook uses an example of the data which is synthetic, but that fits the format of the data derived from the experiments. The goal is to see how the scripts convert the data from the compressed format to an RDF graph and turtle file.

In [1]:
# import our functions from src
from src import *

In [2]:
#import libraries
import yaml
import logging 
import sys
from rdflib import Graph
import os

# Configure logging

logger = logging.getLogger(__name__)

In [3]:
# paths for the data
eln_export = 'example.zip'
output_file = 'tensile_test.ttl'
keymap_file_path = 'keymaps/tensile_test.yml'
plot = True  # whether or not to plot the RDF graph
pattern = 'ftw.json'
loglevel = 'INFO'

In [4]:
# Load the keymap
try:
    with open(keymap_file_path, 'r') as f:
        keymap = yaml.safe_load(f)
        logger.info(f"Keymap loaded from {keymap_file_path}")
except Exception as e:
    logger.error(f"An error occurred while loading the keymap: {e}")
    sys.exit(1)

try:
    # Create a fresh RDF graph each time
    data_graph = Graph()

    # Bind the prefixes to the RDF graph
    bind_prefixes_to_graph(data_graph, keymap['namespaces'])

    # Process each file in the ELN export
    for file_name, json_data in json_data_from_zip_generator(eln_export, pattern):
        process_data_with_mapping(data_graph, json_data, keymap)

    # Serialize the graph to a Turtle file
    data_graph.serialize(destination=output_file, format='turtle')
    logger.info(f"RDF graph serialized to {output_file}")

    # Plot RDF graph if plot is set to True
    if plot:
        image_filename = os.path.splitext(output_file)[0] + '.png'
        plot_rdf_graph(data_graph, image_filename)
        logger.info(f"RDF graph plotted and saved as {image_filename}")

except Exception as e:
    logger.error(f"An error occurred during data generation/processing: {e}")
    sys.exit(1)